In [ ]:
import sys
import os
import random
import torch
import numpy as np
import itertools
import pandas as pd
import csv
from tqdm import tqdm
from box import Box

# Ajustar path al proyecto
project_root = "/home/javitrucas/TFG"
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.dataset_loader import load_dataset
from scripts.medical_scripts.medical_training import Training
from scripts.medical_scripts.medical_evaluation import ModelEvaluator
from scripts.MIL_utils import MIL_collate_fn

def run_experiment(config: Box):
    # Fijar semillas
    seed = config.seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Crear directorio de salida
    out_dir = f"./models/{config.dataset_name.split('-')[0]}/{config.pooling_type}/seed{seed}"
    os.makedirs(out_dir, exist_ok=True)

    # Cargar datos
    train_ds, val_ds = load_dataset(config=config, mode="train_val")
    test_ds          = load_dataset(config=config, mode="test")
    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=config.batch_size,
                                               shuffle=True,  collate_fn=MIL_collate_fn)
    val_loader   = torch.utils.data.DataLoader(val_ds,   batch_size=config.batch_size,
                                               shuffle=False, collate_fn=MIL_collate_fn)
    test_loader  = torch.utils.data.DataLoader(test_ds,  batch_size=config.batch_size,
                                               shuffle=False, collate_fn=MIL_collate_fn)

    # Entrenamiento
    trainer = Training(
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=config.num_epochs,
        learning_rate=config.learning_rate,
        output_model_dir=out_dir,
        pooling_type=config.pooling_type,
        input_feature_dim=config.input_feature_dim,
        feature_dim=config.feature_dim,
        wandb=None
    )
    trainer.train()

    # Número de épocas entrenadas
    # Ajusta `trainer.current_epoch` por el atributo que uses internamente
    trained_epochs = getattr(trainer, 'current_epoch', config.num_epochs)

    # Guardar modelo
    model_path = os.path.join(out_dir, "model.pth")
    torch.save(trainer.model.state_dict(), model_path)

    # Evaluación en train
    evaluator_train = ModelEvaluator(
        model_path=model_path,
        test_loader=train_loader,
        batch_size=config.batch_size,
        input_feature_dim=config.input_feature_dim,
        feature_dim=config.feature_dim,
        pooling_type=config.pooling_type,
        wandb=None
    )
    train_metrics, _ = evaluator_train.evaluate()
    print(f"Train AUC: {train_metrics.get('auc')}")

    # Evaluación en test
    evaluator_test = ModelEvaluator(
        model_path=model_path,
        test_loader=test_loader,
        batch_size=config.batch_size,
        input_feature_dim=config.input_feature_dim,
        feature_dim=config.feature_dim,
        pooling_type=config.pooling_type,
        wandb=None
    )
    test_metrics, _ = evaluator_test.evaluate()
    print(f"Test AUC: {test_metrics.get('auc')}")

    # Devolver métricas + épocas
    return {
        "trained_epochs": trained_epochs,
        **{f"train_{k}": v for k, v in train_metrics.items()},
        **{f"test_{k}":  v for k, v in test_metrics.items()}
    }


def run_all_experiments():
    csv_file = "RSNA_experiment_runs.csv"
    header = [
        "dataset", "pooling", "learning_rate", "feature_dim", "seed",
        "trained_epochs",
        # métricas de train
        "train_loss", "train_accuracy", "train_f1_score", "train_auc", "train_precision", "train_recall",
        # métricas de test
        "test_loss",  "test_accuracy",  "test_f1_score",  "test_auc",  "test_precision",  "test_recall"
    ]

    # Cargar progreso previo
    if os.path.exists(csv_file):
        df_done = pd.read_csv(csv_file)
        print("Últimos experimentos:")
        display(df_done.tail())
        done_set = set(zip(
            df_done.dataset, df_done.pooling,
            df_done.learning_rate, df_done.feature_dim, df_done.seed
        ))
    else:
        with open(csv_file, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)
        done_set = set()
        print("No se encontró CSV previo. Empezando desde cero.")

    # Rejilla de hiperparámetros
    datasets       = ["rsna-features_resnet18"]
    pooling_types  = ["attention", "mean", "max"]
    learning_rates = [1e-3, 1e-4]
    feature_dims   = [64, 128]
    seeds          = list(range(5))

    for dataset, pooling, lr, feat_dim in tqdm(
        itertools.product(datasets, pooling_types, learning_rates, feature_dims),
        desc="Grid search RSNA"
    ):
        for seed in seeds:
            key = (dataset, pooling, lr, feat_dim, seed)
            if key in done_set:
                continue

            # Parámetros dinámicos
            input_feat_dim = 512 if "rsna" in dataset else 1024
            num_epochs     = 50  if "rsna" in dataset else 15

            config = Box({
                "dataset_name":      dataset,
                "input_feature_dim": input_feat_dim,
                "feature_dim":       feat_dim,
                "pooling_type":      pooling,
                "num_epochs":        num_epochs,
                "learning_rate":     lr,
                "batch_size":        1,
                "val_prop":          0.2,
                "seed":              seed,
                "use_inst_distances": False,
                "adj_mat_mode":      "relative"
            })

            print(f"\n=== Ejecutando: {dataset}, {pooling}, lr={lr}, feat_dim={feat_dim}, seed={seed} ===")
            metrics = run_experiment(config)

            # Preparar fila
            row = [
                dataset, pooling, lr, feat_dim, seed,
                metrics.get("trained_epochs", ""),
                metrics.get("train_loss", ""),
                metrics.get("train_accuracy", ""),
                metrics.get("train_f1_score", ""),
                metrics.get("train_auc", ""),
                metrics.get("train_precision", ""),
                metrics.get("train_recall", ""),
                metrics.get("test_loss", ""),
                metrics.get("test_accuracy", ""),
                metrics.get("test_f1_score", ""),
                metrics.get("test_auc", ""),
                metrics.get("test_precision", ""),
                metrics.get("test_recall", ""),
            ]

            # Guardar inmediatamente
            with open(csv_file, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(row)

            done_set.add(key)

    print("\nTodos los experimentos quedan registrados en", csv_file)

if __name__ == "__main__":
    run_all_experiments()


Últimos experimentos:


,dataset,pooling,learning_rate,feature_dim,seed,trained_epochs,train_loss,train_accuracy,train_f1_score,train_auc,train_precision,train_recall,test_loss,test_accuracy,test_f1_score,test_auc,test_precision,test_recall
55,rsna-features_resnet18,max,0.0001,128,0,50,NaN,0.93500,0.920973,0.973070,0.920973,0.920973,NaN,0.866667,0.859155,0.899573,0.871429,0.847222
56,rsna-features_resnet18,max,0.0001,128,1,50,NaN,0.97625,0.971429,0.991256,0.961310,0.981763,NaN,0.866667,0.850746,0.921652,0.919355,0.791667
57,rsna-features_resnet18,max,0.0001,128,2,50,NaN,0.95500,0.945122,0.983202,0.948012,0.942249,NaN,0.840000,0.820896,0.903312,0.887097,0.763889
58,rsna-features_resnet18,max,0.0001,128,3,50,NaN,0.97625,0.971429,0.996631,0.961310,0.981763,NaN,0.846667,0.841379,0.904202,0.835616,0.847222
59,rsna-features_resnet18,max,0.0001,128,4,50,NaN,0.96500,0.958457,0.994657,0.936232,0.981763,NaN,0.833333,0.829932,0.913640,0.813333,0.847222


Grid search RSNA: 12it [00:00, 160803.99it/s]


Todos los experimentos quedan registrados en RSNA_experiment_runs.csv


In [4]:
import pandas as pd
import numpy as np

# Cargar el CSV
df = pd.read_csv('RSNA_experiment_runs.csv')

# Definir las métricas para calcular estadísticas
metrics_to_analyze = [
    'train_accuracy', 'train_f1_score', 'train_auc', 'train_precision', 'train_recall',
    'test_accuracy', 'test_f1_score', 'test_auc', 'test_precision', 'test_recall'
]

# Agrupar por hiperparámetros (excluyendo seed)
groupby_cols = ['dataset', 'pooling', 'learning_rate', 'feature_dim']

# Calcular estadísticas agrupadas
results_summary = []

for group_name, group_data in df.groupby(groupby_cols):
    dataset, pooling, lr, feature_dim = group_name
    
    # Crear diccionario base para este grupo
    row = {
        'dataset': dataset,
        'pooling': pooling,
        'learning_rate': lr,
        'feature_dim': feature_dim,
        'n_seeds': len(group_data)
    }
    
    # Calcular medias y desviaciones para cada métrica
    for metric in metrics_to_analyze:
        if metric in group_data.columns:
            values = group_data[metric].dropna()
            if len(values) > 0:
                row[f'{metric}_mean'] = values.mean()
                row[f'{metric}_std'] = values.std() if len(values) > 1 else 0.0
    
    results_summary.append(row)

# Convertir a DataFrame
df_clean = pd.DataFrame(results_summary)

# Guardar CSV limpio
df_clean.to_csv('RSNA_clean.csv', index=False, float_format='%.6f')

print(f"CSV limpio guardado en 'RSNA_clean.csv' con {len(df_clean)} configuraciones")

CSV limpio guardado en 'RSNA_clean.csv' con 12 configuraciones
